In [49]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np
from tenacity import retry, stop_after_attempt, wait_fixed
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import boxscoreadvancedv3, teamgamelog
import requests

In [37]:
# Example: Fetching all games for a particular season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23')
games = gamefinder.get_data_frames()[0]
game_data = games[['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL']].copy(deep=True)
home_team_data = game_data[game_data['MATCHUP'].str.contains('vs.')].copy(deep=True)
away_team_data = game_data[game_data['MATCHUP'].str.contains('@')].copy(deep=True)
games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42022,1610612743,DEN,Denver Nuggets,0042200405,2023-06-12,DEN vs. MIA,W,240,94,...,0.565,11,46,57,21,6,7,14,13,5.0
1,42022,1610612748,MIA,Miami Heat,0042200405,2023-06-12,MIA @ DEN,L,240,89,...,0.875,11,33,44,18,9,7,8,21,-5.0
2,42022,1610612743,DEN,Denver Nuggets,0042200404,2023-06-09,DEN @ MIA,W,242,108,...,0.762,5,29,34,26,11,7,6,18,13.0
3,42022,1610612748,MIA,Miami Heat,0042200404,2023-06-09,MIA vs. DEN,L,240,95,...,0.850,8,29,37,23,2,3,14,19,-13.0
4,42022,1610612743,DEN,Denver Nuggets,0042200403,2023-06-07,DEN @ MIA,W,238,109,...,0.815,13,45,58,28,3,5,13,18,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4313,12022,1610612749,MIL,Milwaukee Bucks,0012200003,2022-10-01,MIL vs. MEM,L,240,102,...,0.714,17,33,50,22,10,3,23,29,-5.0
4314,12022,50009,MRA,Ra'anana Maccabi Ra'anana,0012200002,2022-09-30,MRA @ LAC,L,240,81,...,0.710,13,25,38,16,14,5,16,34,-43.2
4315,12022,1610612744,GSW,Golden State Warriors,0012200001,2022-09-30,GSW @ WAS,W,239,96,...,0.775,10,52,62,18,6,3,17,27,9.0
4316,12022,1610612746,LAC,LA Clippers,0012200002,2022-09-30,LAC vs. MRA,W,242,121,...,0.698,15,40,55,27,9,6,23,23,44.2


In [38]:
home_team_data.rename(columns={'TEAM_ABBREVIATION': 'H_TEAM', "WL" : "H_TEAM_WIN"}, inplace=True)
# encode win/loss as 1/0
# home_team_data['H_TEAM_WIN'] = home_team_data['H_TEAM_WIN'].replace('W', 1)
# home_team_data['H_TEAM_WIN'] = home_team_data['H_TEAM_WIN'].replace('L', 0)

home_team_data

,SEASON_ID,TEAM_ID,H_TEAM,GAME_ID,GAME_DATE,MATCHUP,H_TEAM_WIN
0,42022,1610612743,DEN,0042200405,2023-06-12,DEN vs. MIA,W
3,42022,1610612748,MIA,0042200404,2023-06-09,MIA vs. DEN,L
5,42022,1610612748,MIA,0042200403,2023-06-07,MIA vs. DEN,L
6,42022,1610612743,DEN,0042200402,2023-06-04,DEN vs. MIA,L
9,42022,1610612743,DEN,0042200401,2023-06-01,DEN vs. MIA,W
...,...,...,...,...,...,...,...
4307,12022,1610612761,TOR,0012200006,2022-10-02,TOR vs. UTA,W
4309,12022,1610612745,HOU,0012200007,2022-10-02,HOU vs. SAS,W
4313,12022,1610612749,MIL,0012200003,2022-10-01,MIL vs. MEM,L
4316,12022,1610612746,LAC,0012200002,2022-09-30,LAC vs. MRA,W


In [39]:
away_team_data.rename(columns={'TEAM_ABBREVIATION': 'A_TEAM'}, inplace=True)
# away_team_data.drop(columns=['WL'], inplace=True)
away_team_data

,SEASON_ID,TEAM_ID,A_TEAM,GAME_ID,GAME_DATE,MATCHUP,WL
1,42022,1610612748,MIA,0042200405,2023-06-12,MIA @ DEN,L
2,42022,1610612743,DEN,0042200404,2023-06-09,DEN @ MIA,W
4,42022,1610612743,DEN,0042200403,2023-06-07,DEN @ MIA,W
7,42022,1610612748,MIA,0042200402,2023-06-04,MIA @ DEN,W
8,42022,1610612748,MIA,0042200401,2023-06-01,MIA @ DEN,L
...,...,...,...,...,...,...,...
4310,12022,1610612762,UTA,0012200006,2022-10-02,UTA @ TOR,L
4311,12022,1610612759,SAS,0012200007,2022-10-02,SAS @ HOU,L
4312,12022,1610612763,MEM,0012200003,2022-10-01,MEM @ MIL,W
4314,12022,50009,MRA,0012200002,2022-09-30,MRA @ LAC,L


In [40]:
from nba_api.stats.endpoints import leaguegamelog
def season_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        season_w = past_games['WL'].value_counts()['W']
        # season_l = past_games['WL'].value_counts()['L']
        # season_w = season_w / (season_w + season_l)
    except KeyError:
        season_w = 0
    return season_w

def last10_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last10_w = past_games['WL'].head(10).value_counts()['W']
        # last10_l = past_games['WL'].head(10).value_counts()['L']
        # last10_w = last10_w / (last10_w + last10_l)
    except KeyError:
        last10_w = 0
    return last10_w

def last5_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last5_w = past_games['WL'].head(5).value_counts()['W']
        # last5_l = past_games['WL'].head(5).value_counts()['L']
        # last5_w = last5_w / (last5_w + last5_l)
    except KeyError:
        last5_w = 0
    return last5_w

def season_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        season_w_opp = past_games[past_mathups]['WL'].value_counts()['W']
        # season_l_opp = past_games[past_mathups]['WL'].value_counts()['L']
        # season_w_opp = season_w_opp / (season_w_opp + season_l_opp)
    except KeyError:
        season_w_opp = 0
    return season_w_opp

def last10_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last10_w_opp = past_games[past_mathups]['WL'].head(10).value_counts()['W']
        # last10_l_opp = past_games[past_mathups]['WL'].head(10).value_counts()['L']
        # last10_w_opp = last10_w_opp / (last10_w_opp + last10_l_opp)
    except KeyError:
        last10_w_opp = 0
    return last10_w_opp

def last5_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last5_w_opp = past_games[past_mathups]['WL'].head(5).value_counts()['W']
        # last5_l_opp = past_games[past_mathups]['WL'].head(5).value_counts()['L']
        # last5_w_opp = last5_w_opp / (last5_w_opp + last5_l_opp)
    except KeyError:
        last5_w_opp = 0
    return last5_w_opp

def avg_pts(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_pts = past_games.head(10)['PTS'].mean()
    return avg_pts

home_team_data["H_SSN_W"] = home_team_data.apply(season_w, axis=1)
home_team_data["H_LAST10_W"] = home_team_data.apply(last10_w, axis=1)
home_team_data["H_LAST5_W"] = home_team_data.apply(last5_w, axis=1)
home_team_data["H_SEASON_W_OPP"] = home_team_data.apply(season_w_opp, axis=1)
home_team_data["H_LAST10_W_OPP"] = home_team_data.apply(last10_w_opp, axis=1)
home_team_data["H_LAST5_W_OPP"] = home_team_data.apply(last5_w_opp, axis=1)
home_team_data["H_AVG_PTS"] = home_team_data.apply(avg_pts, axis=1)
home_team_data

# advanced_boxscore = requests.get("https://stats.nba.com/stats/boxscoreadvancedv3?EndPeriod=1&EndRange=0&GameID=0021700807&RangeType=0&StartPeriod=1&StartRange=0")
# advanced_boxscore = advanced_boxscore.get_data_frames()[1]
# advanced_boxscore


,SEASON_ID,TEAM_ID,H_TEAM,GAME_ID,GAME_DATE,MATCHUP,H_TEAM_WIN,H_SSN_W,H_LAST10_W,H_LAST5_W,H_SEASON_W_OPP,H_LAST10_W_OPP,H_LAST5_W_OPP,H_AVG_PTS
0,42022,1610612743,DEN,0042200405,2023-06-12,DEN vs. MIA,W,71,9,4,5,5,4,114.4
3,42022,1610612748,MIA,0042200404,2023-06-09,MIA vs. DEN,L,62,5,2,1,1,1,106.2
5,42022,1610612748,MIA,0042200403,2023-06-07,MIA vs. DEN,L,62,6,2,1,1,1,106.4
6,42022,1610612743,DEN,0042200402,2023-06-04,DEN vs. MIA,L,69,8,5,3,3,3,115.4
9,42022,1610612743,DEN,0042200401,2023-06-01,DEN vs. MIA,W,68,8,5,2,2,2,117.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,12022,1610612761,TOR,0012200006,2022-10-02,TOR vs. UTA,W,0,0,0,0,0,0,NaN
4309,12022,1610612745,HOU,0012200007,2022-10-02,HOU vs. SAS,W,0,0,0,0,0,0,NaN
4313,12022,1610612749,MIL,0012200003,2022-10-01,MIL vs. MEM,L,0,0,0,0,0,0,NaN
4316,12022,1610612746,LAC,0012200002,2022-09-30,LAC vs. MRA,W,0,0,0,0,0,0,NaN


In [36]:
boxscore_data = pd.read_csv('adv_bosxscore3.csv')

def net_rating(row):
    last10_games = games[(games['TEAM_ABBREVIATION'] == row['H_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])].head(10)
    sum = 0
    for i in range(len(last10_games)):
        relevant_game = boxscore_data[(boxscore_data['gameId'] == last10_games.iloc[i]['GAME_ID']) & (boxscore_data['teamId'] == row['TEAM_ID'])]
        print(len(relevant_game))
        sum = sum + relevant_game['offensiveRating'] - relevant_game['defensiveRating']
        # print(type(sum))
    return sum / 10

home_team_data["NET_RATING"] = home_team_data.apply(net_rating, axis=1)


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [41]:
def season_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        season_w = past_games['WL'].value_counts()['W']
        # season_l = past_games['WL'].value_counts()['L']
        # season_w = season_w / (season_w + season_l)
    except KeyError:
        season_w = 0
    return season_w

def last10_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last10_w = past_games['WL'].head(10).value_counts()['W']
        # last10_l = past_games['WL'].head(10).value_counts()['L']
        # last10_w = last10_w / (last10_w + last10_l)
    except KeyError:
        last10_w = 0
    return last10_w

def last5_w(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    try:
        last5_w = past_games['WL'].head(5).value_counts()['W']
        # last5_l = past_games['WL'].head(5).value_counts()['L']
        # last5_w = last5_w / (last5_w + last5_l)
    except KeyError:
        last5_w = 0
    return last5_w

def season_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        season_w_opp = past_games[past_mathups]['WL'].value_counts()['W']
        # season_l_opp = past_games[past_mathups]['WL'].value_counts()['L']
        # season_w_opp = season_w_opp / (season_w_opp + season_l_opp)
    except KeyError:
        season_w_opp = 0
    return season_w_opp

def last10_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last10_w_opp = past_games[past_mathups]['WL'].head(10).value_counts()['W']
        # last10_l_opp = past_games[past_mathups]['WL'].head(10).value_counts()['L']
        # last10_w_opp = last10_w_opp / (last10_w_opp + last10_l_opp)
    except KeyError:
        last10_w_opp = 0
    return last10_w_opp

def last5_w_opp(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    opp_team = row['MATCHUP'].split(' ')[2]
    past_mathups = past_games['MATCHUP'].str.contains(opp_team)
    try:
        last5_w_opp = past_games[past_mathups]['WL'].head(5).value_counts()['W']
        # last5_l_opp = past_games[past_mathups]['WL'].head(5).value_counts()['L']
        # last5_w_opp = last5_w_opp / (last5_w_opp + last5_l_opp)
    except KeyError:
        last5_w_opp = 0
    return last5_w_opp

def avg_pts(row):
    past_games = games[(games['TEAM_ABBREVIATION'] == row['A_TEAM']) & (games['GAME_DATE'] < row['GAME_DATE'])]
    avg_pts = past_games.head(10)['PTS'].mean()
    return avg_pts

away_team_data["A_SSN_W"] = away_team_data.apply(season_w, axis=1)
away_team_data["A_LAST10_W"] = away_team_data.apply(last10_w, axis=1)
away_team_data["A_LAST5_W"] = away_team_data.apply(last5_w, axis=1)
away_team_data["A_SSN_W_OPP"] = away_team_data.apply(season_w_opp, axis=1)
away_team_data["A_LAST10_W_OPP"] = away_team_data.apply(last10_w_opp, axis=1)
away_team_data["A_LAST5_W_OPP"] = away_team_data.apply(last5_w_opp, axis=1)
away_team_data["A_AVG_PTS"] = away_team_data.apply(avg_pts, axis=1)
away_team_data


,SEASON_ID,TEAM_ID,A_TEAM,GAME_ID,GAME_DATE,MATCHUP,WL,A_SSN_W,A_LAST10_W,A_LAST5_W,A_SSN_W_OPP,A_LAST10_W_OPP,A_LAST5_W_OPP,A_AVG_PTS
1,42022,1610612748,MIA,0042200405,2023-06-12,MIA @ DEN,L,62,4,2,1,1,1,103.4
2,42022,1610612743,DEN,0042200404,2023-06-09,DEN @ MIA,W,70,8,4,4,4,4,116.0
4,42022,1610612743,DEN,0042200403,2023-06-07,DEN @ MIA,W,69,7,4,3,3,3,116.5
7,42022,1610612748,MIA,0042200402,2023-06-04,MIA @ DEN,W,61,5,1,0,0,0,105.6
8,42022,1610612748,MIA,0042200401,2023-06-01,MIA @ DEN,L,61,6,2,0,0,0,107.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4310,12022,1610612762,UTA,0012200006,2022-10-02,UTA @ TOR,L,0,0,0,0,0,0,NaN
4311,12022,1610612759,SAS,0012200007,2022-10-02,SAS @ HOU,L,0,0,0,0,0,0,NaN
4312,12022,1610612763,MEM,0012200003,2022-10-01,MEM @ MIL,W,0,0,0,0,0,0,NaN
4314,12022,50009,MRA,0012200002,2022-09-30,MRA @ LAC,L,0,0,0,0,0,0,NaN


## combining home and away team data

In [44]:
# drop useless columns
home_team_data.drop(columns=['SEASON_ID', 'TEAM_ID', 'GAME_DATE', 'MATCHUP'], inplace=True)
away_team_data.drop(columns=['SEASON_ID', 'TEAM_ID', 'GAME_DATE', 'MATCHUP', 'WL'], inplace=True)


KeyError: "['SEASON_ID', 'TEAM_ID', 'GAME_DATE', 'MATCHUP'] not found in axis"

In [48]:
# merge home and away data around game id
game_data = pd.merge(home_team_data, away_team_data, left_on='GAME_ID', right_on='GAME_ID')
# remove game id row
game_data.drop(columns=['GAME_ID'], inplace=True)

#replace NaNs with 0
game_data = game_data.fillna(0)
game_data.to_csv('game_data.csv')
game_data

,H_TEAM,H_TEAM_WIN,H_SSN_W,H_LAST10_W,H_LAST5_W,H_SEASON_W_OPP,H_LAST10_W_OPP,H_LAST5_W_OPP,H_AVG_PTS,A_TEAM,A_SSN_W,A_LAST10_W,A_LAST5_W,A_SSN_W_OPP,A_LAST10_W_OPP,A_LAST5_W_OPP,A_AVG_PTS
0,DEN,W,71,9,4,5,5,4,114.4,MIA,62,4,2,1,1,1,103.4
1,MIA,L,62,5,2,1,1,1,106.2,DEN,70,8,4,4,4,4,116.0
2,MIA,L,62,6,2,1,1,1,106.4,DEN,69,7,4,3,3,3,116.5
3,DEN,L,69,8,5,3,3,3,115.4,MIA,61,5,1,0,0,0,105.6
4,DEN,W,68,8,5,2,2,2,117.5,MIA,61,6,2,0,0,0,107.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,TOR,W,0,0,0,0,0,0,0.0,UTA,0,0,0,0,0,0,0.0
2155,HOU,W,0,0,0,0,0,0,0.0,SAS,0,0,0,0,0,0,0.0
2156,MIL,L,0,0,0,0,0,0,0.0,MEM,0,0,0,0,0,0,0.0
2157,LAC,W,0,0,0,0,0,0,0.0,MRA,0,0,0,0,0,0,0.0


## Collecting Advanced Box score data

In [7]:
from nba_api.stats.endpoints import boxscoreadvancedv3
advanced_boxscore3 = pd.DataFrame()

def advanced_boxscore_applier(row, advanced_boxscore3):
    try:
        advanced_boxscore = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=row['GAME_ID'])
        advanced_boxscore = advanced_boxscore.get_data_frames()[1]
        advanced_boxscore3 = pd.concat([advanced_boxscore3, advanced_boxscore])
        return advanced_boxscore3
    except TimeoutError:
        return advanced_boxscore3
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return advanced_boxscore3

for i in range(2100, 2159):
    advanced_boxscore3 = advanced_boxscore_applier(home_team_data.iloc[i], advanced_boxscore3)
advanced_boxscore3.to_csv('2100-2149_adv_bosxscore3.csv')
advanced_boxscore3

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,minutes,estimatedOffensiveRating,offensiveRating,estimatedDefensiveRating,...,turnoverRatio,effectiveFieldGoalPercentage,trueShootingPercentage,usagePercentage,estimatedUsagePercentage,estimatedPace,pace,pacePer40,possessions,PIE
0,0012200057,1610612747,Los Angeles,Lakers,LAL,lakers,240:00,98.0,99.1,104.2,...,17.5,0.506,0.569,1.0,0.200,114.24,114.0,95.00,114.0,0.490
1,0012200057,1610612750,Minnesota,Timberwolves,MIN,timberwolves,240:00,104.2,103.5,98.0,...,10.5,0.495,0.535,1.0,0.200,114.24,114.0,95.00,114.0,0.510
0,0012200054,1610612755,Philadelphia,76ers,PHI,sixers,240:00,96.4,98.0,91.3,...,11.9,0.455,0.502,1.0,0.199,102.80,101.0,84.17,101.0,0.510
1,0012200054,1610612766,Charlotte,Hornets,CHA,hornets,240:00,91.3,93.1,96.4,...,23.8,0.525,0.541,1.0,0.198,102.80,101.0,84.17,101.0,0.490
0,0012200053,1610612754,Indiana,Pacers,IND,pacers,240:00,105.3,105.8,93.7,...,16.5,0.529,0.564,1.0,0.199,105.12,103.0,85.83,103.0,0.588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,0012200003,1610612763,Memphis,Grizzlies,MEM,grizzlies,240:00,105.1,105.9,100.8,...,21.8,0.558,0.596,1.0,0.197,101.50,101.5,84.58,101.0,0.576
0,0012200002,1610612746,LA,Clippers,LAC,clippers,240:00,115.3,117.5,78.9,...,23.3,0.591,0.631,1.0,0.198,103.78,104.0,86.67,103.0,0.796
1,0012200002,50009,Ra'anana,Maccabi Ra'anana,MRA,maccabi-ra'anana,240:00,78.9,77.1,115.3,...,16.2,0.347,0.411,1.0,0.197,103.78,104.0,86.67,105.0,0.204
0,0012200001,1610612764,Washington,Wizards,WAS,wizards,240:00,83.5,82.9,92.7,...,13.3,0.405,0.448,1.0,0.200,103.90,105.5,87.92,105.0,0.441


In [ ]:
endpoints import boxscoreadvancedv3
advanced_boxscore3 = pd.DataFrame()

def advanced_boxscore_applier(row, advanced_boxscore3):
    try:
        advanced_boxscore = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=row['GAME_ID'])
        advanced_boxscore = advanced_boxscore.get_data_frames()[1]
        advanced_boxscore3 = pd.concat([advanced_boxscore3, advanced_boxscore])
        return advanced_boxscore3
    except TimeoutError:
        return advanced_boxscore3
    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return advanced_boxscore3

for i in range(2100, 2159):
    advanced_boxscore3 = advanced_boxscore_applier(home_team_data.iloc[i], advanced_boxscore3)
advanced_boxscore3.to_csv('2100-2149_adv_bosxscore3.csv')
advanced_boxscore3

In [9]:
csv_files = ['0-999_adv_bosxscore3.csv', '1000-2150_adv_bosxscore3.csv']
dataframes = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_csv('adv_bosxscore3.csv')

In [7]:
games_2024 = leaguegamefinder.LeagueGameFinder(season_nullable='2024-25')
games_2024 = games_2024.get_data_frames()[0]
games_2024

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
